<a id="title_ID"></a>

<a href="http://photutils.readthedocs.io/en/stable/index.html"><img src="https://photutils.readthedocs.io/en/stable/_static/photutils_banner.svg" width=300></a>

# PSF Photometry with `photutils`
---

##### What is PSF photometry?
A more specific form of photometry than aperture photometry, PSF photometry takes into account the shape of a source's point spread function (PSF). The PSF is a model that represents the distribution of light from a point source as it falls onto a detector. An example of a basic PSF is simply a 2-D Gaussian, while more complex PSFs can include distortion, diffraction, or interference effects associated with a particular telescope. For instance, the PSFs from the Hubble Space Telescope and the James Webb Space Telescope have been meticulously modeled, and can be simulated with the [Tiny Tim](http://www.stsci.edu/hst/observatory/focus/TinyTim) and [WebbPSF](https://github.com/mperrin/webbpsf) software packages, respectively. However, for datasets that do not have readily available PSF models, such models can be statistically generated by analyzing the image itself.

The `photutils` package provides tools that combine background estimation, source detection, and model-fitting to perform PSF photometry on image data.

##### What does this tutorial include?
This tutorial covers how to perform PSF photometry with `photutils`, including the following methods:
* Gaussian PSF Photometry
* Iterative Subtraction
* Point Response Function (PRF) Photometry

##### Which data are used in this tutorial?
We will be manipulating Hubble eXtreme Deep Field (XDF) data, which was collected using the Advanced Camera for Surveys (ACS) on Hubble between 2002 and 2012. The image we use here is the result of 1.8 million seconds (500 hours!) of exposure time, and includes some of the faintest and most distant galaxies that have ever been observed. 

*The methods demonstrated here are available in narrative form within the `photutils.psf` [documentation](http://photutils.readthedocs.io/en/stable/psf.html).*

<div class="alert alert-block alert-danger">
    
**Warning:** The PSF photometry API is currently considered experimental and may change in the future. The photutils development team will aim to keep compatibility where practical, but will not finalize the API until sufficient user feedback has been accumulated.

</div>

<div class="alert alert-block alert-warning">
    
<b>Important:</b> Before proceeding, please be sure to update your versions of <code>astropy</code>, <code>matplotlib</code>, and <code>photutils</code>, or this notebook may not work properly. Or, if you don't want to handle packages individually, you can always use (and keep updated!) the <a href="https://astroconda.readthedocs.io">AstroConda</a> distribution.
 
</div>

---

## Import necessary packages

First, let's import packages that we will use to perform arithmetic functions and visualize data:

In [ ]:
from astropy.io import fits
import astropy.units as u
from astropy.nddata import CCDData
# from astropy.stats import sigma_clipped_stats
from astropy.visualization import ImageNormalize, LogStretch
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
import numpy as np

# Show plots in the notebook
%matplotlib inline

Let's also define some `matplotlib` parameters, such as title font size and the dpi, to make sure our plots look nice. To make it quick, we'll do this by loading a [style file shared with the other photutils tutorials](../photutils_notebook_style.mplstyle) into `pyplot`. We will use this style file for all the notebook tutorials. (See [here](https://matplotlib.org/users/customizing.html) to learn more about customizing `matplotlib`.)

In [ ]:
plt.style.use('../photutils_notebook_style.mplstyle')

## Retrieve data

As described in the introduction, we will be using Hubble eXtreme Deep Field (XDF) data. Since this file is too large to store on GitHub, we will just use `astropy` to directly download the file from the STScI archive: https://archive.stsci.edu/prepds/xdf/ 

(Generally, the best package for web queries of astronomical data is [Astroquery](https://astroquery.readthedocs.io/en/latest/); however, the dataset we are using is a High Level Science Product (HLSP) and thus is not located within a catalog that could be queried with Astroquery.)

In [ ]:
url = 'https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f435w_v1_sci.fits'
with fits.open(url) as hdulist:
    hdulist.info()
    data = hdulist[0].data
    header = hdulist[0].header

As explained in a [previous notebook](../01_background_estimation/01_background_estimation.ipynb) on background estimation, it is important to **mask** these data, as a large portion of the values are equal to zero. We will mask out the non-data portions of the image array, so all of those pixels that have a value of zero don't interfere with our statistics and analyses of the data. 

In [ ]:
# Define the mask
mask = data == 0

Throughout this notebook, we are going to store our images in Python using a `CCDData` object (see [Astropy documentation](http://docs.astropy.org/en/stable/nddata/index.html#ccddata-class-for-images)), which contains a `numpy` array in addition to metadata such as uncertainty, masks, or units. In this case, our data is in electrons (counts) per second.

In [ ]:
unit = u.electron / u.s
xdf_image = CCDData(data, unit=unit, meta=header, mask=mask)

Let's look at the data:

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Set up the normalization and colormap
norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
cmap = plt.get_cmap('viridis')
cmap.set_over(cmap.colors[-1])
cmap.set_under(cmap.colors[0])
cmap.set_bad('white') # Show masked data as white
xdf_image_clipped = np.clip(xdf_image, 1e-4, None) # clip to plot with logarithmic stretch

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)

# Define the colorbar and fix the labels
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')

*Tip: Double-click on any inline plot to zoom in.*

## Circular Apertures

### Creating Apertures

### Performing Aperture Photometry

### Calculating Aperture Corrections

## Elliptical Apertures

### Creating Apertures

### Performing Aperture Photometry

### Calculating Aperture Corrections

---
## Exercises

---
## Additional Resources
For more examples and details, please visit the [photutils](http://photutils.readthedocs.io/en/stable/index.html) documentation.

---
## About this Notebook
**Authors:** Lauren Chambers (lchambers@stsci.edu)
<br>**Updated:** May 2019

[Top of Page](#title_ID)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>